In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/chat_bot/


Mounted at /content/drive
/content/drive/MyDrive/chat_bot


In [120]:
import json
import numpy as np
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [156]:
with open('intents.json') as file:
    data = json.load(file)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return words


In [147]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

In [157]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        processed_pattern = preprocess_text(pattern)
        training_sentences.append(' '.join(processed_pattern))
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])


In [158]:
label_encoder = LabelEncoder()
training_labels = label_encoder.fit_transform(training_labels)


In [159]:
vocab_size = 2000
max_len = 20

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')


In [160]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout
from tensorflow.keras.optimizers import Adam


In [161]:
model = Sequential()
model.add(Embedding(vocab_size, 16, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

history = model.fit(padded_sequences, np.array(training_labels),
                    epochs=300, verbose=1)


Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.0155 - loss: 3.6376
Epoch 2/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0632 - loss: 3.6294
Epoch 3/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0770 - loss: 3.6210
Epoch 4/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0581 - loss: 3.6062
Epoch 5/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0638 - loss: 3.5843
Epoch 6/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0917 - loss: 3.5381
Epoch 7/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0560 - loss: 3.5438
Epoch 8/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0675 - loss: 3.5207
Epoch 9/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0648 - loss: 3.4942
Epoch 10/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0913 - loss: 3.5079
Epoch 11/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0422 - loss: 3.5128
Epoch 12/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0831 - lo

In [99]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification

with open('intents.json') as f:
    data = json.load(f)

texts = []
labels = []
label_map = {}
for idx, intent in enumerate(data['intents']):
    label_map[intent['tag']] = idx
    for pattern in intent['patterns']:
        texts.append(pattern)
        labels.append(idx)

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding='max_length',
                            max_length=128, return_tensors="pt")
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length',
                          max_length=128, return_tensors="pt")

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


In [101]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)


In [102]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    fp16=True,
    label_smoothing_factor=0.1)



In [103]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
10,3.576600,3.645814
20,3.620700,3.643646
30,3.639100,3.636702
40,3.696400,3.631667
50,3.679400,3.620124
60,3.663200,3.609043
70,3.665900,3.597828
80,3.688100,3.591213
90,3.613900,3.601631
100,3.636400,3.566852


TrainOutput(global_step=405, training_loss=3.039959356519911, metrics={'train_runtime': 4209.1899, 'train_samples_per_second': 0.385, 'train_steps_per_second': 0.096, 'total_flos': 106594420746240.0, 'train_loss': 3.039959356519911, 'epoch': 5.0})

In [104]:
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")


Evaluation results: {'eval_loss': 1.943200945854187, 'eval_runtime': 45.6651, 'eval_samples_per_second': 1.774, 'eval_steps_per_second': 0.46, 'epoch': 5.0}


In [127]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        processed_input = preprocess_text(inp)
        seq = tokenizer.texts_to_sequences([' '.join(processed_input)])
        padded_seq = pad_sequences(seq, maxlen=max_len, truncating='post')

        prediction = model.predict(padded_seq)
        tag = label_encoder.inverse_transform([np.argmax(prediction)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(f"Chatbot: {np.random.choice(i['responses'])}")


In [130]:
chat()


Start talking with the bot (type quit to stop)!
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Hi there, how can I help?
You: what is the timing of school?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Chatbot: College is open 8am-5pm Monday-Saturday!
You: what is in canteen?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Chatbot: Our university has canteen with variety of food available
You: what are the food options?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: Our university has canteen with variety of food available
You: damn
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Chatbot: Hello!
You: nice
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: I am glad I helped you
You: how are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Chatbot: Hi there, how can I help?
You: how many program offered?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.
You: quit


In [146]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_text(text):
    text = text.lower()
    return text.split()

def chat_response(inp):
    processed_input = preprocess_text(inp)
    seq = tokenizer.texts_to_sequences([' '.join(processed_input)])
    padded_seq = pad_sequences(seq, maxlen=max_len, truncating='post')

    prediction = model.predict(padded_seq)
    tag = label_encoder.inverse_transform([np.argmax(prediction)])[0]

    for i in data['intents']:
        if i['tag'] == tag:
            return np.random.choice(i['responses'])

iface = gr.Interface(
    fn=chat_response,
    inputs="text",
    outputs="text",
    title="Chatbot",
    description="Start chatting with the bot. Type your message and press enter.")

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c30b8fc0c03e144693.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
